In [7]:
import pandas as pd
import numpy as np
import os
#config data, saved to config.py. Change values as instructed
from config import postgresUname,postgresPword,postgresHost,postgresDb

### Extract CSVs into DataFrames

In [8]:
tweets_file = os.path.join("Resources", "trumptweets.csv")


In [9]:
approval_file = os.path.join("Resources", "approval.csv")


In [12]:
#Load tweets to data frame
tweets_df = pd.read_csv(tweets_file)
tweets_df.head()

,Unnamed: 0,date,target,insult,tweet
0,1,2014-10-09,thomas-frieden,fool,"Can you believe this fool, Dr. Thomas Frieden ..."
1,2,2014-10-09,thomas-frieden,DOPE,"Can you believe this fool, Dr. Thomas Frieden ..."
2,3,2015-06-16,politicians,all talk and no action,Big time in U.S. today - MAKE AMERICA GREAT AG...
3,4,2015-06-24,ben-cardin,It's politicians like Cardin that have destroy...,Politician @SenatorCardin didn't like that I s...
4,5,2015-06-24,neil-young,total hypocrite,"For the nonbeliever, here is a photo of @Neily..."


In [11]:
#Load approval polls to data frame
approval_df = pd.read_csv(approval_file)
approval_df.head()

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Donald Trump,All polls,1/20/2021,1/20/2017,1/22/2017,Morning Consult,B/C,1992.0,rv,0.680029,...,37.0,45.686784,38.055805,NaN,NaN,http://static.politico.com/9b/13/82a3baf542ae9...,49249,77261,1/23/2017,11:47:59 20 Jan 2021
1,Donald Trump,All polls,1/20/2021,1/20/2017,1/22/2017,Gallup,B,1500.0,a,0.262323,...,45.0,45.861441,43.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49253,77265,1/23/2017,11:47:59 20 Jan 2021
2,Donald Trump,All polls,1/20/2021,1/20/2017,1/24/2017,Ipsos,B-,1632.0,a,0.153481,...,45.2,43.451563,43.780389,NaN,T,http://polling.reuters.com/#poll/CP3_2/,49426,77599,3/1/2017,11:47:59 20 Jan 2021
3,Donald Trump,All polls,1/20/2021,1/21/2017,1/23/2017,Gallup,B,1500.0,a,0.242845,...,46.0,45.861441,44.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49262,77274,1/24/2017,11:47:59 20 Jan 2021
4,Donald Trump,All polls,1/20/2021,1/22/2017,1/24/2017,Gallup,B,1500.0,a,0.227380,...,45.0,46.861441,43.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49236,77248,1/25/2017,11:47:59 20 Jan 2021


### Transform tweets DataFrame

In [20]:
#Delete all tweets before 2017 inauguration
cleanTweets_df = tweets_df.drop(tweets_df[tweets_df['date'] < '2017-01-20'].index)
#delete column thats not needed
del cleanTweets_df['Unnamed: 0']
cleanTweets_df.head()

,date,target,insult,tweet
2357,2017-01-25,cnn,FAKE NEWS,Congratulations to @FoxNews for being number o...
2358,2017-01-25,chicago,If Chicago doesn't fix the horrible carnage go...,"If Chicago doesn't fix the horrible ""carnage"" ..."
2359,2017-01-26,chelsea-manning,Ungrateful TRAITOR,"Ungrateful TRAITOR Chelsea Manning, who should..."
2360,2017-01-26,chelsea-manning,should never have been released from prison,"Ungrateful TRAITOR Chelsea Manning, who should..."
2361,2017-01-26,chelsea-manning,Terrible!,"Ungrateful TRAITOR Chelsea Manning, who should..."


In [21]:
#Copy only relevant data to a new data frame
tweetdates_df = cleanTweets_df[["date", "target"]].copy()
tweetdates_df.head()

,date,target
2357,2017-01-25,cnn
2358,2017-01-25,chicago
2359,2017-01-26,chelsea-manning
2360,2017-01-26,chelsea-manning
2361,2017-01-26,chelsea-manning


In [22]:
#Testing to see how many tweets per day
grouped_tweets = tweetdates_df.groupby(by="date").count()
grouped_tweets.head()

,target
date,
2017-01-25,2
2017-01-26,4
2017-01-28,8
2017-01-29,5
2017-01-31,6


### Transform approval DataFrame

In [8]:
#Shrinking approval polls to relevant fields for app (url to be used as reference if required)
approvalfinal_df = approval_df[["startdate","enddate","pollster","samplesize","adjusted_approve","adjusted_disapprove","url"]].copy()
approvalfinal_df.head()

,startdate,enddate,pollster,samplesize,adjusted_approve,adjusted_disapprove,url
0,1/20/2017,1/22/2017,Morning Consult,1992.0,45.686784,38.055805,http://static.politico.com/9b/13/82a3baf542ae9...
1,1/20/2017,1/22/2017,Gallup,1500.0,45.861441,43.539189,http://www.gallup.com/poll/201617/gallup-daily...
2,1/20/2017,1/24/2017,Ipsos,1632.0,43.451563,43.780389,http://polling.reuters.com/#poll/CP3_2/
3,1/21/2017,1/23/2017,Gallup,1500.0,45.861441,44.539189,http://www.gallup.com/poll/201617/gallup-daily...
4,1/22/2017,1/24/2017,Gallup,1500.0,46.861441,43.539189,http://www.gallup.com/poll/201617/gallup-daily...


In [9]:
#Converting dates to ISO format
approvalfinal_df["startdate"] = pd.to_datetime(approvalfinal_df.startdate)
approvalfinal_df["enddate"] = pd.to_datetime(approvalfinal_df.enddate)
approvalfinal_df.head()

,startdate,enddate,pollster,samplesize,adjusted_approve,adjusted_disapprove,url
0,2017-01-20,2017-01-22,Morning Consult,1992.0,45.686784,38.055805,http://static.politico.com/9b/13/82a3baf542ae9...
1,2017-01-20,2017-01-22,Gallup,1500.0,45.861441,43.539189,http://www.gallup.com/poll/201617/gallup-daily...
2,2017-01-20,2017-01-24,Ipsos,1632.0,43.451563,43.780389,http://polling.reuters.com/#poll/CP3_2/
3,2017-01-21,2017-01-23,Gallup,1500.0,45.861441,44.539189,http://www.gallup.com/poll/201617/gallup-daily...
4,2017-01-22,2017-01-24,Gallup,1500.0,46.861441,43.539189,http://www.gallup.com/poll/201617/gallup-daily...


### Create database connection

In [ ]:
#connection string using config data
rds_connection_string = f'{postgresUname}:{postgresPword}@{postgresHost}/{postgresDb}'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

### Load DataFrames into database

In [ ]:
#load tweet data
tweetdates_df.to_sql(name='Trumptweets', con=engine, if_exists='append', index=False)

In [ ]:
#load approval data
approvalfinal_df.to_sql(name='Approval_rating', con=engine, if_exists='append', index=False)

###  Confirm data has been added

In [ ]:
#check tweet data
pd.read_sql_query('select * from Trumptweets', con=engine).head()

In [ ]:
#check approval data
pd.read_sql_query('select * from Approval_rating', con=engine).head()